# Um Sistema de Recomendação da Movie Lens

## Introdução

O dataset [MovieLens](https://grouplens.org/datasets/movielens/) é um conjunto de dados que contém várias classificações (ratings) de filmes por usuários. Este conjunto de dados é frequentemente usado para experimentos de pesquisa e prototipagem em sistemas de recomendação. O projeto MovieLens foi criado pelo GroupLens, um laboratório de pesquisa na University of Minnesota, e serve como uma das referências padrão na área de sistemas de recomendação.

O conjunto de dados tem várias versões, algumas contendo apenas algumas dezenas de milhares de classificações e outras contendo até 20 milhões ou mais. Cada registro geralmente contém:

    ID do usuário que deu a classificação
    ID do filme classificado
    A classificação dada (geralmente em uma escala de 1 a 5)
    Um atributo timestamp indicando quando a classificação foi dada

Algumas versões também incluem informações adicionais, como tags atribuídas aos filmes, gêneros, e até mesmo links para dados relacionados, como imagens de capas de filmes ou metadados. Por ser um conjunto de dados bem estruturado e extensivo, o MovieLens é amplamente utilizado para demonstrar algoritmos de recomendação, desde métodos simples, como a Filtragem Colaborativa e a Filtragem Baseada em Conteúdo, até técnicas mais avançadas como Sistemas de Recomendação baseados em Aprendizado Profundo.

## Baixando o dataset

In [ ]:
# No Colab não é necessário executar esse comando
# %pip install gdown

In [ ]:
# Importa o dataset de filmes
!gdown 1ovr90WWjeLh_PWqe5yLIK-1aIzHvtuDk

In [ ]:
# Import o dataset com as avaliações dos usuários
!gdown 1pN2Upg6J7mie1esMREFD5rHjZOpk9QXb

## Preparando o dataset para o treinamento

**Bibliotecas**

In [25]:
import pandas as pd
import numpy as np

### `movies.dat`

**Testando o acesso ao dataset**

In [26]:
# Adapte o nome do caminho para usar o caderno no Colab
df_filmes = pd.read_csv('./movies.dat', sep='::', engine='python', names=['id_filme', 'nome', 'categoria'])
df_filmes.head()

,id_filme,nome,categoria
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
print(df_filmes.id_filme.nunique())

### `ratings.csv`

**Testando o acesso ao dataset de ratings**

In [27]:
#  Adapte o nome do caminho para usar o caderno no Colab
df_ava = pd.read_csv(
        './ratings_mini.csv', # local e nome do arquivo
        sep= ',', # Separador
        dtype={'id_usuario': np.int32, 'id_filme': np.int32, 'avaliacao': np.float64, 'timestamp':np.int32}) # Define o tipo de cada coluna
df_ava.head()

,id_usuario,id_filme,avaliacao,timestamp
0,5,1,1.0,857911264
1,5,7,3.0,857911357
2,5,25,3.0,857911265
3,5,28,3.0,857913507
4,5,30,5.0,857911752


In [ ]:
print(df_ava.id_usuario.nunique())

# Há filmes que não foram avaliados
print(df_ava.id_filme.nunique())

In [ ]:
print(len(df_ava))

## Criando sistemas de recomendação

### Filtragem Colaborativa

A filtragem colaborativa faz recomendações com base em padrões de comportamento passado de vários usuários, sem necessitar de qualquer informação adicional sobre os itens ou usuários. No caso do SVD, ele tenta capturar padrões latentes nas avaliações dos usuários para fazer previsões para outros itens que o usuário ainda não avaliou. A ideia básica da filtragem colaborativa é criar uma matriz usuário-item. O conjunto de dados pode ser representado como uma matriz onde as linhas correspondem aos usuários e as colunas aos filmes.


 Matrix | Inception | Titanic | Star Wars | The Godfather
--------|-----------|---------|-----------|--------------
Alice    |     5     |    3    |     4     |      0
Bob      |     4     |    0    |     5     |      3
Carol    |     3     |    5    |     4     |      4
Dave     |     0     |    2    |     0     |      5
Eve      |     2     |    5    |     0     |      4


Em algoritmos como o $kNN$ temos que fazer esse passo manualmente. O `scikit-surprise` converte esse formato longo em uma matriz usuário-item internamente para executar algoritmos como Single Value Decomposition (SVD).

In [ ]:
%pip install surprise

In [28]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import cross_validate

#### Treinamento do modelo de SVD

In [29]:
# As avaliações vão de 1 a 5 no dataset
reader = Reader(rating_scale=(1, 5))

# Carrega os dados para um formato que o SVD consegue ler
data = Dataset.load_from_df(df_ava[['id_usuario', 'id_filme', 'avaliacao']], reader)

# Cria um modelo SVD
model = SVD()

# Realizando validação cruzada
resultados_cv = cross_validate(model, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

# Construindo conjunto de treinamento
trainset = data.build_full_trainset()

# Ajusta o modelo SVD
model.fit(trainset)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8263  0.8267  0.8244  0.8271  0.8262  0.8262  0.0009  
MAE (testset)     0.6353  0.6353  0.6336  0.6361  0.6352  0.6351  0.0008  
Fit time          62.32   70.03   65.53   79.58   79.29   71.35   7.04    
Test time         18.99   15.68   14.10   29.67   15.02   18.69   5.73    


**[Ex01]** Interprete o resultado do treinamento a seguir. O que a média (*Mean*) do RMSE e do MAE querem dizer?

```
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8263  0.8267  0.8244  0.8271  0.8262  0.8262  0.0009  
MAE (testset)     0.6353  0.6353  0.6336  0.6361  0.6352  0.6351  0.0008  
Fit time          62.32   70.03   65.53   79.58   79.29   71.35   7.04    
Test time         18.99   15.68   14.10   29.67   15.02   18.69   5.73
```    

Escreva sua resposta aqui.

*Clique duas vezes aqui para ver a resposta*

<!--
O resultado mostra a avaliação de desempenho do algoritmo de Fatoração de Matrizes Singular (SVD, Singular Value Decomposition) aplicado a um conjunto de dados do Movie Lens. Esta avaliação foi realizada usando validação cruzada com 5 folds. Vamos analisar cada métrica:
RMSE (Root Mean Square Error)

O RMSE é uma medida que captura a média das diferenças quadráticas entre as avaliações reais e as previsões. Valores menores de RMSE indicam melhores previsões. No contexto de sistemas de recomendação, um RMSE baixo sugere que o sistema é bom em prever as avaliações que os usuários dariam aos filmes.

* Fold 1, Fold 2, ..., Fold 5: Estes são os RMSEs calculados para cada um dos 5 conjuntos de teste na validação cruzada.
* Mean: A média dos RMSEs é de aproximadamente 0.8262, indicando que, em média, o erro de previsão é de cerca de 0.8262 pontos (na escala de avaliação).
* Std: O desvio padrão é 0.0009, o que é relativamente baixo, indicando que o algoritmo teve um desempenho consistente nos diferentes conjuntos de teste.

MAE (Mean Absolute Error)

O MAE é outra métrica que mede o erro entre as previsões e as avaliações reais. No entanto, ao contrário do RMSE, ele usa o valor absoluto da diferença em vez do quadrado. Um MAE baixo indica boas previsões.

* Mean: A média do MAE é de aproximadamente 0.6351, o que indica que, em média, o erro absoluto entre a previsão e a avaliação real é de cerca de 0.6351 pontos.
* Std: O desvio padrão é 0.0008, o que também é baixo, sugerindo desempenho consistente.
-->

#### Fazendo previsões com o modelo

In [30]:
user_id = 7
filme_id = 5 

model.predict(user_id, filme_id)

Prediction(uid=7, iid=5, r_ui=None, est=2.5322582793130977, details={'was_impossible': False})

Intepretando esse resultado, temos que:

* `uid`: ID do usuário

* `iid`: ID do item (no nosso caso, filmes)

* `r_ui`: Avaliação real data pelo usuário ao item. O valor `None` sugere que o usuário não avaliou esse item

* `est`: Previsão da avaliação do usuário.

* `details={'was_impossible': False}`: Este campo fornece informações adicionais sobre a previsão. O campo `was_impossible` indica se a previsão poderia ser feita ou não. Neste caso, é False, o que significa que a previsão foi possível.

#### Testando o modelo para um usuário específico

**[Ex02]**. Complete o código a seguir de tal forma que seja feita uma previsão para todos os filmes que um determinado usuário não tenha assistido.

In [31]:
user_id = 7 # Um usuário qualquer

# Encontrar todos os filmes únicos no dataset
todos_filmes = df_ava['id_filme'].unique()

# Encontrar os filmes que o usuário já avaliou
filmes_avaliados = # Complete aqui

# Filmes que o usuário não assistiu ainda
filmes_nao_assistidos = # Complete aqui

# Fazendo previsões para os filmes não assistidos
predicoes = # Complete aqui

# Ordenando as previsões
predicoes_ordenadas = sorted(predicoes, key=lambda x: x.est, reverse=True)

# Mostrando as recomendações para o usuário user_id
print(f"Recomendações para o usuário {user_id}:")
for pred in predicoes_ordenadas:
    print(f"id_filme: {# complete aqui}, avaliacao_estimada: {# Complete aqui}")

Recomendações para o usuário 7:
id_filme: 3134, avaliacao_estimada: 4.938521246778224
id_filme: 750, avaliacao_estimada: 4.851861988876151
id_filme: 858, avaliacao_estimada: 4.842075920144308
id_filme: 1230, avaliacao_estimada: 4.794740237009225
id_filme: 2731, avaliacao_estimada: 4.748752250838166
id_filme: 6918, avaliacao_estimada: 4.745092291107208
id_filme: 1204, avaliacao_estimada: 4.733723456266547
id_filme: 1281, avaliacao_estimada: 4.7077404404563135
id_filme: 922, avaliacao_estimada: 4.697147476410935
id_filme: 1221, avaliacao_estimada: 4.69039110426643
id_filme: 3089, avaliacao_estimada: 4.687915067610431
id_filme: 3030, avaliacao_estimada: 4.680659960285882
id_filme: 1193, avaliacao_estimada: 4.668354338371507
id_filme: 2351, avaliacao_estimada: 4.663618696159313
id_filme: 1178, avaliacao_estimada: 4.662056133151064
id_filme: 969, avaliacao_estimada: 4.651516858270253
id_filme: 1267, avaliacao_estimada: 4.649421819616777
id_filme: 3545, avaliacao_estimada: 4.6452688423406014

*Clique duas vezes aqui para ver a resposta*

<!--
user_id = 7 # Um usuário qualquer

# Encontrar todos os filmes únicos no dataset
todos_filmes = df_ava['id_filme'].unique()

# Encontrar os filmes que o usuário já avaliou
filmes_avaliados = df_ava[df_ava['id_usuario'] == user_id]['id_filme'].tolist()

# Filmes que o usuário não assistiu ainda
filmes_nao_assistidos = [filme for filme in todos_filmes if filme not in filmes_avaliados]

# Fazendo previsões para os filmes não assistidos
predicoes = [model.predict(user_id, filme) for filme in filmes_nao_assistidos]

# Ordenando as previsões
predicoes_ordenadas = sorted(predicoes, key=lambda x: x.est, reverse=True)

# Mostrando as recomendações para o usuário user_id
print(f"Recomendações para o usuário {user_id}:")
for pred in predicoes_ordenadas:
    print(f"id_filme: {pred.iid}, avaliacao_estimada: {pred.est}")
-->

#### Desenvolvendo uma função de recomendação

**[Ex03]**. Complete a função a seguir para que sejam feitas `n_recomendacoes` a um determinado usuário.

In [ ]:
def recommend_top_n_movies(user_id, model, df_ava, df_filmes, n_recomendacoes=5):
    # Encontrar todos os filmes únicos no dataset
    todos_filmes = df_ava['id_filme'].unique()
    
    # Encontrar os filmes que o usuário já avaliou
    filmes_avaliados = # Complete aqui
    
    # Filmes que o usuário não assistiu ainda
    filmes_nao_assistidos = # Complete aqui

    # Fazendo previsões para os filmes não assistidos
    predicoes = # Complete aqui
    
    # Ordenando as previsões em ordem decrescente (ou não crescente)
    predicoes_ordenadas = # Complete aqui
    
    # Retornar apenas as top 'n_recomendacoes' recomendações
    top_preds = # Complete aqui
    
    # Lista de retorno
    recomendacoes = []
    for pred in top_preds:
        nome_filme = # Complete aqui
        
        # Armazena as informações em um formato de dicionário que poderia ser útil para converter para JSON
        recomendacoes.append({"id_filme": pred.iid, "nome": nome_filme, "avaliacao_estimada": pred.est})

    return recomendacoes

# Uso da função
user_id = 7
recommend_top_n_movies(user_id, model, df_ava, df_filmes)

*Clique duas vezes aqui para ler a resposta*

<!--
def recommend_top_n_movies(user_id, model, df_ava, df_filmes, n_recomendacoes=5):
    # Encontrar todos os filmes únicos no dataset
    todos_filmes = df_ava['id_filme'].unique()
    
    # Encontrar os filmes que o usuário já avaliou
    filmes_avaliados = df_ava[df_ava['id_usuario'] == user_id]['id_filme'].tolist()
    
    # Filmes que o usuário não assistiu ainda
    filmes_nao_assistidos = [filme for filme in todos_filmes if filme not in filmes_avaliados]
    
    # Fazendo previsões para os filmes não assistidos
    predicoes = [model.predict(user_id, filme) for filme in filmes_nao_assistidos]
    
    # Ordenando as previsões em ordem decrescente (ou não crescente)
    predicoes_ordenadas = sorted(predicoes, key=lambda x: x.est, reverse=True)
    
    # Retornar apenas as top 'n_recomendacoes' recomendações
    top_preds = predicoes_ordenadas[:n_recomendacoes]
    
    # Lista de retorno
    recomendacoes = []
    for pred in top_preds:
        nome_filme = df_filmes[df_filmes['id_filme'] == pred.iid]['nome'].iloc[0]
        # Armazena as informações em um formato de dicionário que poderia ser útil para converter para JSON
        recomendacoes.append({"id_filme": pred.iid, "nome": nome_filme, "avaliacao_estimada": pred.est})

    return recomendacoes

# Uso da função
user_id = 7
recommend_top_n_movies(user_id, model, df_ava, df_filmes)
-->

### Filtragem por Conteúdo

A filtragem por conteúdo é uma técnica de sistema de recomendação que usa as características do item e um perfil do usuário para sugerir itens específicos. Ao contrário da filtragem colaborativa, que se baseia nas interações entre usuários e itens, a filtragem por conteúdo se concentra nos atributos do próprio item, como gênero, diretor, ou palavras-chave, e nas preferências do usuário.

#### Similaridade do coseno

A similaridade do coseno é uma métrica usada para determinar o grau de similaridade entre dois vetores. É especialmente útil em sistemas de recomendação para encontrar a similaridade entre usuários ou itens. A ideia é tratar as preferências de cada usuário como um vetor em um espaço n-dimensional e calcular o cosseno do ângulo entre esses vetores. Um valor de similaridade do coseno de 1 significa que os vetores são idênticos, e um valor de -1 significa que são completamente opostos.

Vamos ver um exemplo concreto. Suponha que temos dois usuários, Alice e Bob, que avaliaram três filmes diferentes. Os filmes são de gêneros de Ação, Comédia e Romance. As avaliações vão de 1 a 5, e elas podem ser vistas como vetores:

- Alice deu as seguintes avaliações: [Ação: 5, Comédia: 3, Romance: 1]
- Bob deu as seguintes avaliações: [Ação: 2, Comédia: 4, Romance: 5]

O vetor de Alice seria $A = [5, 3, 1]$ e o vetor de Bob seria $B = [2, 4, 5]$.

Primeiro, calculamos o produto escalar ($ A \cdot B $):

$ A \cdot B = (5 * 2) + (3 * 4) + (1 * 5) = 10 + 12 + 5 = 27 $

Depois, calculamos as normas de $ A $ e $ B $:

$ \|A\| = \sqrt{(5^2) + (3^2) + (1^2)} = \sqrt{25 + 9 + 1} = \sqrt{35} $

$ \|B\| = \sqrt{(2^2) + (4^2) + (5^2)} = \sqrt{4 + 16 + 25} = \sqrt{45} $

Finalmente, a similaridade do coseno entre Alice e Bob é:

$ \text{Similaridade do Cosseno} = \frac{27}{\sqrt{35} \times \sqrt{45}} \approx 0.65 $

Esse valor de $0.65$ indica uma similaridade moderada entre Alice e Bob com base em suas avaliações de filmes. Essa métrica pode ser usada em um sistema de recomendação para sugerir filmes que um usuário similar gostou.

#### Preparando o dataset de filmes

Vamos expandir a coluna categoria em múltiplas colunas, mas antes vamos verificar se há filmes sem classificação de categoria.

In [5]:
# Investigando um pouco, vemos que essa linha está sem classificação de categoria
df_filmes.iloc[7903]

id_filme                     8606
nome         Pull My Daisy (1958)
categoria      (no genres listed)
Name: 7903, dtype: object

In [6]:
# Vamos removê-la para simplificar o processo
df_filmes.drop(7903, inplace=True)

In [7]:
# Dividir a coluna 'categoria' em múltiplas colunas, uma para cada categoria
categoria_dummies = df_filmes['categoria'].str.get_dummies(sep='|')

# Juntar o DataFrame original com o novo DataFrame de categorias
df_filmes = pd.concat([df_filmes, categoria_dummies], axis=1)

# Descartar a coluna 'categoria', uma vez que ela foi one-hot-encoded
df_filmes.drop('categoria', axis=1, inplace=True)

# Exibir o DataFrame resultante
print(df_filmes)

       id_filme                                nome  Action  Adventure  \
0             1                    Toy Story (1995)       0          1   
1             2                      Jumanji (1995)       0          1   
2             3             Grumpier Old Men (1995)       0          0   
3             4            Waiting to Exhale (1995)       0          0   
4             5  Father of the Bride Part II (1995)       0          0   
...         ...                                 ...     ...        ...   
10676     65088              Bedtime Stories (2008)       0          1   
10677     65091          Manhattan Melodrama (1934)       0          0   
10678     65126                        Choke (2008)       0          0   
10679     65130           Revolutionary Road (2008)       0          0   
10680     65133      Blackadder Back & Forth (1999)       0          0   

       Animation  Children  Comedy  Crime  Documentary  Drama  ...  Film-Noir  \
0              1         1    

In [ ]:
print(len(df_filmes))

#### Combinando os datasets em um só

In [22]:
df_ava.columns

Index(['id_usuario', 'id_filme', 'avaliacao', 'timestamp'], dtype='object')

In [23]:
df_filmes.columns

Index(['id_filme', 'nome', 'Action', 'Adventure', 'Animation', 'Children',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir',
       'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller',
       'War', 'Western'],
      dtype='object')

In [18]:
df = pd.merge(df_ava, df_filmes, on='id_filme')
df.head()

,id_usuario,id_filme,avaliacao,timestamp,nome,Action,Adventure,Animation,Children,Comedy,...,Film-Noir,Horror,IMAX,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,5,1,1.0,857911264,Toy Story (1995),0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,14,1,3.0,1133572007,Toy Story (1995),0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,30,1,5.0,876526432,Toy Story (1995),0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
3,35,1,3.0,1133601653,Toy Story (1995),0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,43,1,4.0,912611519,Toy Story (1995),0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0


#### Construindo um perfil de usuário

O perfil de usuário é uma representação computacional das preferências de um usuário individual. Ele é calculado com base nas avaliações que o usuário deu para diferentes tipos de conteúdo, neste caso, gêneros de filmes. O perfil serve como um resumo ponderado das inclinações do usuário em direção a diferentes gêneros.

A ideia é que, ao entender as preferências de gênero de um usuário, o sistema pode fazer recomendações melhores. Por exemplo, se o perfil indica uma forte preferência por filmes de ação e uma baixa preferência por romances, o sistema pode priorizar a recomendação de novos filmes de ação.

Vamos ilustrar isso com um exemplo simples para entender como o perfil de um usuário é construído.

Suponha que um usuário, chamado Bob, assistiu e avaliou três filmes:

1. Um filme de Ação com avaliação 5.
2. Um filme de Comédia com avaliação 4.
3. Outro filme de Ação com avaliação 3.

Seu DataFrame para o usuário Bob ficaria assim:

| id_usuario | avaliacao | Action | Comedy |
|------------|-----------|--------|--------|
|     Bob    |     5     |   1    |   0    |
|     Bob    |     4     |   0    |   1    |
|     Bob    |     3     |   1    |   0    |

Para calcular o perfil de Bob para o gênero Ação, faríamos o seguinte cálculo:

Perfil de Ação de Bob = ((5 * 1) + (4 * 0) + (3 * 1)) / (5 + 4 + 3)
                      = (5 + 0 + 3) / 12
                      = 8 / 12
                      = 0.667

Para o gênero Comédia:

Perfil de Comédia de Bob = ((5 * 0) + (4 * 1) + (3 * 0)) / (5 + 4 + 3)
                         = (0 + 4 + 0) / 12
                         = 4 / 12
                         = 0.333

Então, o perfil de Bob seria:

- Ação: 0.667
- Comédia: 0.333

Este perfil é armazenado em um dicionário, onde a chave é "Bob" e o valor é outro dicionário contendo suas preferências por gênero. É basicamente isso que o código faz, mas para todos os usuários do DataFrame.

**[Ex04]** Complete o código onde indicado.

In [19]:
# Selecionar as colunas relacionadas a gêneros de filmes no DataFrame
df_cols_generos = df.loc[:, 'Action':'Western']

# Inicializar um dicionário para armazenar o perfil de cada usuário
perfil_usuario = {}

# Iterar sobre cada usuário único no DataFrame
for usuario in df['id_usuario'].unique():
    # Filtrar as linhas do DataFrame que correspondem ao usuário atual
    usuario_df = df[df['id_usuario'] == usuario]
    
    # Inicializar um dicionário para armazenar o perfil do usuário atual
    perfil = {}
    
    # Iterar sobre cada gênero de filme
    for genero in df_cols_generos.columns:
        # Calcular a média ponderada das avaliações para cada gênero
        # Multiplica a coluna do gênero pela coluna de avaliação e soma os resultados
        # Em seguida, divide pela soma total das avaliações do usuário
        
        perfil[genero] = # Complete aqui
    
    # Armazenar o perfil calculado no dicionário de perfis de usuário
    perfil_usuario[usuario] = perfil

# Mostrar o dicionário de perfis de usuário
perfil_usuario


{5: {'Action': 0.026865671641791045,
  'Adventure': 0.07164179104477612,
  'Animation': 0.0029850746268656717,
  'Children': 0.03880597014925373,
  'Comedy': 0.31044776119402984,
  'Crime': 0.14029850746268657,
  'Documentary': 0.0,
  'Drama': 0.764179104477612,
  'Fantasy': 0.04477611940298507,
  'Film-Noir': 0.014925373134328358,
  'Horror': 0.07761194029850746,
  'IMAX': 0.0,
  'Musical': 0.04477611940298507,
  'Mystery': 0.050746268656716415,
  'Romance': 0.29253731343283584,
  'Sci-Fi': 0.08059701492537313,
  'Thriller': 0.16716417910447762,
  'War': 0.07164179104477612,
  'Western': 0.0},
 14: {'Action': 0.19359756097560976,
  'Adventure': 0.35365853658536583,
  'Animation': 0.12195121951219512,
  'Children': 0.28810975609756095,
  'Comedy': 0.5777439024390244,
  'Crime': 0.0350609756097561,
  'Documentary': 0.001524390243902439,
  'Drama': 0.27896341463414637,
  'Fantasy': 0.2865853658536585,
  'Film-Noir': 0.0,
  'Horror': 0.013719512195121951,
  'IMAX': 0.0,
  'Musical': 0.170

*Clique duas vezes aqui para ler a resposta*

<!-- 
# Selecionar as colunas relacionadas a gêneros de filmes no DataFrame
df_cols_generos = df.loc[:, 'Action':'Western']

# Inicializar um dicionário para armazenar o perfil de cada usuário
perfil_usuario = {}

# Iterar sobre cada usuário único no DataFrame
for usuario in df['id_usuario'].unique():
    # Filtrar as linhas do DataFrame que correspondem ao usuário atual
    usuario_df = df[df['id_usuario'] == usuario]
    
    # Inicializar um dicionário para armazenar o perfil do usuário atual
    perfil = {}
    
    # Iterar sobre cada gênero de filme
    for genero in df_cols_generos.columns:
        # Calcular a média ponderada das avaliações para cada gênero
        # Multiplica a coluna do gênero pela coluna de avaliação e soma os resultados
        # Em seguida, divide pela soma total das avaliações do usuário
        perfil[genero] = (usuario_df[genero] * usuario_df['avaliacao']).sum() / usuario_df['avaliacao'].sum()
    
    # Armazenar o perfil calculado no dicionário de perfis de usuário
    perfil_usuario[usuario] = perfil

# Mostrar o dicionário de perfis de usuário
perfil_usuario
-->

In [32]:
from sklearn.metrics.pairwise import cosine_similarity

# Calcular a similaridade e classificar os filmes
user_id = 10  # exemplo
perfil_alvo = pd.DataFrame([perfil_usuario[user_id]])  # perfil do usuário 1 como DataFrame
filmes = df[df_cols_generos.columns].drop_duplicates()  # apenas características únicas dos filmes

# Calculando similaridades
similaridades = cosine_similarity(perfil_alvo, filmes)
recomendacoes = pd.DataFrame({'id_filme': filmes.index, 'similaridade': similaridades[0]})
recomendacoes = recomendacoes.sort_values(by='similaridade', ascending=False)

# Passo 4: Recomendar filmes não assistidos
filmes_assistidos = df[df['id_usuario'] == usuario_alvo]['id_filme'].tolist()
recomendacoes_filtradas = recomendacoes[~recomendacoes['id_filme'].isin(filmes_assistidos)]

print("Recomendações para o usuário:", usuario_alvo)
print(recomendacoes_filtradas)

: 

: 

**[Ex05]** Escreva uma função de recomendação para o código anterior.

*Resposta desse exercício não está inclusa*